## Import Libraries

In [22]:
# First pandas and numpy libraries are imported.
import pandas as pd
import numpy as np

## Clean Data

In [23]:
# Next, the csv files containing test results are read and stored in dataframes. 
# The csv files are accessed from the TestResult folder.
df_Test1 = pd.read_csv('TestResult/Formative_Test_1.csv')
df_Test2 = pd.read_csv('TestResult/Formative_Test_2.csv')
df_Test3 = pd.read_csv('TestResult/Formative_Test_3.csv')
df_Test4 = pd.read_csv('TestResult/Formative_Test_4.csv')
df_MockTest = pd.read_csv('TestResult/Formative_Mock_Test.csv')
df_SumTest = pd.read_csv('TestResult/SumTest.csv')


In [24]:
def CleanTest(df):
    """ Cleans data frame:
        df = dataframe(pandas.core.frame.DataFrame)."""
    
    # Make a copy of the data frame.
    df_copy = df.copy()
    
    # Remove space between column names.
    df_copy.columns = df_copy.columns.str.replace(' ', '') 
    
    # Remove mark information from question column names.
    df_copy.columns = df_copy.columns.str.split('/')
    df_copy.columns = [item[0] for item in df_copy.columns]
    
    # Replace all missing values with zero.
    df_copy = df_copy.replace('-', 0)
    
    # Sort researchid and grade, and if there are more than one results for a student, only the highest mark is selected.
    df_copy = df_copy.sort_values(['researchid', 'Grade'],
                                  ascending = (False, False)).groupby('researchid').first()
    
    # Remove the 'State' and 'Timetaken' columns.
    df_copy.drop(['State', 'Timetaken'], axis = 1, inplace = True)
    
    # Return the cleaned data frame.
    return df_copy

# Clean all the test dataframes and assign new variable names.
df_CleanTest1 = CleanTest(df_Test1)
df_CleanTest2 = CleanTest(df_Test2)
df_CleanTest3 = CleanTest(df_Test3)
df_CleanTest4 = CleanTest(df_Test4)
df_CleanMockTest = CleanTest(df_MockTest)
df_CleanSumTest = CleanTest(df_SumTest)

## Format Data

In [25]:
def FormatCleanTest(dfClean, dfRaw):
    """ Normalizes grades in a dataframe:
        dfClean = cleaned data frame(pandas.core.frame.DataFrame)
        dfRaw = original dataframe that has not been cleaned (pandas.core.frame.DataFrame)."""
    
    # Make copies of the clean and original data frames. 
    # The original data frame is needed because it contains the mark information of questions.
    df_Cleancopy = dfClean.copy()
    df_Rawcopy = dfRaw.copy()
    
    # Remove spaces between column names, and split the question name and mark information in the question column names.
    df_Rawcopy.columns = df_Rawcopy.columns.str.replace(' ', '')
    df_Rawcopy.columns = df_Rawcopy.columns.str.split('/')
    
    # Put the Grade and question column names in a list and extract the mark information from the column names
    df_Rawcopy.columns[5:,].tolist()
    mylist = []
    for val in df_Rawcopy.columns[5:,].tolist():
        mylist.append(float(val[1]))
    
    # Normalize the values in each grade and question column by multiplying each value by 10000 and dividing by the mark information.
    df_Cleancopy.loc[:, df_Cleancopy.columns.str.startswith("Q")] = df_Cleancopy.loc[:,df_Cleancopy.columns.str.startswith("Q")].astype(float)*10000/(np.array(mylist[1:]))
    df_Cleancopy['Grade'] = df_Cleancopy['Grade'].astype(float) * 10000/mylist[0]
    
    # Return the normalized dataframe
    return df_Cleancopy

# Format each of the clean test data frames and store them in new variables.
df_FormattedCleanTest1 = FormatCleanTest(df_CleanTest1, df_Test1)
df_FormattedCleanTest2 = FormatCleanTest(df_CleanTest2, df_Test2)
df_FormattedCleanTest3 = FormatCleanTest(df_CleanTest3, df_Test3)
df_FormattedCleanTest4 = FormatCleanTest(df_CleanTest4, df_Test4)
df_FormattedCleanMockTest = FormatCleanTest(df_CleanMockTest, df_MockTest)
df_FormattedCleanSumTest = FormatCleanTest(df_CleanSumTest, df_SumTest)   

## Store Data

In [26]:
# Import sqlite library.
import sqlite3

In [27]:
def StoreData(df, tablename):
    """Stores data frame as an sql table in ResultDatabase:
       df = formatted and cleaned data frame(pandas.core.frame.DataFrame)
       tablename = name of sql table(str)."""
    
    # Count how many questions there are in the data frame.
    questions = df.columns.str.startswith("Q").sum()
    
    # The SQL CREATE TABLE command is written and columns necessary for the data frame storage are created.
    # The string is stored in the sqltable variable.
    sqltable = """ CREATE TABLE IF NOT EXISTS""" + " " + tablename + """(
    researchid INT PRIMARY KEY,
    Startedon CHAR(30),
    Completed CHAR(30),
    Grade FLOAT\n,""" 
    
    for i in range(1,questions + 1):
        if i != questions:
            sqltable = sqltable + "Q"+str(i)+" Float,\n"
        else:
            sqltable = sqltable + "Q"+str(i)+" Float\n"
            
        
    sqltable = sqltable + ');'
    
    # A connection is made to the database and the table is created.
    connection = sqlite3.connect('ResultDatabase.db')
    cursor = connection.cursor()
    cursor.execute(sqltable)
    
    # Save changes and insert the data frame into the sql table.
    connection.commit()
    df.to_sql(tablename, if_exists = 'replace', con=connection)
    
    # Save changes and close the connection.
    connection.commit()
    connection.close()
    
    
# Create a table for each test data frame, and store them in the database.
StoreData(df_FormattedCleanTest1, 'test1')
StoreData(df_FormattedCleanTest2, 'test2')
StoreData(df_FormattedCleanTest3, 'test3')
StoreData(df_FormattedCleanTest4, 'test4')
StoreData(df_FormattedCleanMockTest, 'MockTest')
StoreData(df_FormattedCleanSumTest, 'SumTest')

  
    

## Modify Data

In [28]:
# Set random seed as 3 to ensure reproducibility.
np.random.seed(3)

# Generate 20 random numbers between 0 and 150 and store in x. Generate 3 random numbers between 0 and 13 and store in y.
x = np.random.randint(150, size = 20)
y = np.random.choice(13,3, replace = False)

# Use the random numbers in x to select rows in the formatted SumTest data frame and save as my_df.
my_df = df_FormattedCleanSumTest.iloc[x]

# Make a list that has the column names of the randomly selected questions.
list1 = []
for i in range(0,3):
    list1.append("Q"+str(y[i]))

# Fill the question columns of the extracted rows with random numbers between 0 and 100.
for i in range(0,3):
    my_df.loc[:,list1[i]] = np.random.randint(100, size = len(my_df))

# Calculate the total in the 'Grade' column, with weighted marks from each of the questions.
my_df['Grade'] = sum([my_df['Q1'] * 500, my_df['Q2'] * 300, my_df['Q3'] * 600, my_df['Q4'] * 700, 
                      my_df['Q5'] * 400, my_df['Q6'] * 500,my_df['Q7'] * 1500, my_df['Q8'] * 1500, my_df['Q9'] * 1500,
                      my_df['Q10'] * 1000, my_df['Q11'] * 400, my_df['Q12'] * 500, my_df['Q13'] * 600])/10000

# Update the formatted clean SumTest dataframe with the modified extracted rows and columns.
df_FormattedCleanSumTest.update(my_df)

# Update the table in the ResultDatabase with the updated data frame.
StoreData(df_FormattedCleanSumTest, 'SumTest')


C:\Users\kipye\AppData\Local\Temp/ipykernel_16268/1863607280.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['Grade'] = sum([my_df['Q1'] * 500, my_df['Q2'] * 300, my_df['Q3'] * 600, my_df['Q4'] * 700,
